In [1]:
from numpy import random as rand
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from shapely.geometry import Polygon, Point
from matplotlib import pyplot as plt

In [20]:
nqubits = 4         # num qubits, min 4, always 2**num_layers qubits
num_iters = 60      # Training iterations 
random = False

# Data hyper-parameters
num_data = 15       # How many ground states do we want for training?
perc_train = 0.75   # Percentage of num_data that will be used for training. Max = 1.
batch_size = 5      # batch training size

# Tweak hyper-parameters
max_weight_init = 0.01  # weight_init goes from 0 to this number. Max = 2*np.pi
stepsize = 0.01         # stepsize of the gradient descent.

layers = int(np.log2(nqubits))
nweights = 30*(layers-1) + 15

In [3]:
def X(i):
    return qml.PauliX(i)

def Z(i):
    return qml.PauliZ(i)

# Gound states

In [4]:
def ground_state(j1, j2):
    
    hamiltonian = 0
    for i in range(nqubits):
        hamiltonian += Z(i)
        hamiltonian -= j1 * X(i) @ X((i+1)%nqubits)
        hamiltonian -= j2 * X((i-1)%nqubits) @ Z(i) @ X((i+1)%nqubits)
    
    _, eigvecs = np.linalg.eigh(qml.matrix(hamiltonian))
    
    return eigvecs[:,0]

In [5]:
def labeling(x, y):
    # Definir las coordenadas de los puntos de cada región
    region1_coords = [(-2, 1), (2, 1), (0, -1)]
    region2_coords = [(0, -1), (3, -4), (4, -4), (4, 3)]
    region3_coords = [(0, -1), (-3, -4), (-4, -4), (-4, 3)]
    region4_coords = [(-2, 1), (2, 1), (4, 3), (4, 4), (-4, 4), (-4, 3)]
    region5_coords = [(-3, -4), (0, -1), (3, -4)]
    
    # Crear objetos Polygon para cada región
    region1_poly = Polygon(region1_coords)
    region2_poly = Polygon(region2_coords)
    region3_poly = Polygon(region3_coords)
    region4_poly = Polygon(region4_coords)
    region5_poly = Polygon(region5_coords)
    
    punto = Point(x, y)
    if region1_poly.contains(punto):
        return 3
    elif region2_poly.contains(punto):
        return 1
    elif region3_poly.contains(punto):
        return 2
    elif region4_poly.contains(punto):
        return 0
    elif region5_poly.contains(punto):
        return 0
    else:
        return None # Si el punto no está en ninguna región

In [6]:
if not random:
    rng = rand.RandomState(0)

gs_list = []
labels_list = []
j_list = rng.uniform(-4, 4, (num_data,2))

for i in range(num_data):
    gs_list.append(ground_state(j_list[i,0], j_list[i,1]))
    labels_list.append(labeling(j_list[i,0], j_list[i,1]))

gs_list = np.array(gs_list, requires_grad=False)
labels_list = np.array(labels_list, requires_grad=False)

In [7]:
print(gs_list.shape)

(15, 16)


# CNN

In [9]:
def convolutional_layer(q1, q2, weights):
    qml.U3(wires=q1, theta=weights[0], phi=weights[1], delta=weights[2])
    qml.U3(wires=q1, theta=weights[3], phi=weights[4], delta=weights[5])
    qml.CNOT(wires=[q2, q1])
    qml.RZ(wires=q1, phi=weights[6])
    qml.RY(wires=q2, phi=weights[7])
    qml.CNOT(wires=[q1, q2])
    qml.RY(wires=q2, phi=weights[8])
    qml.CNOT(wires=[q2, q1])
    qml.U3(wires=q1, theta=weights[9], phi=weights[10], delta=weights[11])
    qml.U3(wires=q1, theta=weights[12], phi=weights[13], delta=weights[14])

def pooling_layer(q1, q2, weights):
    qml.U3(wires=q1, theta=weights[0], phi=weights[1], delta=weights[2])
    qml.U3(wires=q1, theta=weights[3], phi=weights[4], delta=weights[5])
    qml.CNOT(wires=[q2, q1])
    qml.RZ(wires=q1, phi=weights[6])
    qml.RY(wires=q2, phi=weights[7])
    qml.CNOT(wires=[q1, q2])
    qml.RY(wires=q2, phi=weights[8])
    qml.CNOT(wires=[q2, q1])
    qml.U3(wires=q1, theta=weights[9], phi=weights[10], delta=weights[11])
    qml.U3(wires=q1, theta=weights[12], phi=weights[13], delta=weights[14])

In [17]:
def cnn_circuit(weights, state_ini):

    qubits = list(range(nqubits))
    
    qml.QubitStateVector(state_ini, wires=qubits)

    for j in range(layers-1):
        
        len_qubits = len(qubits)
        
        for i in range(len_qubits//2):
            convolutional_layer(qubits[2*i], qubits[(2*i+1)%len_qubits], weights[15*2*j:15*(2*j+1)])
        
        for i in range(len_qubits//2):
            convolutional_layer(qubits[2*i+1], qubits[(2*i+2)%len_qubits], weights[15*2*j:15*(2*j+1)])
            
        for i in range(len_qubits//2):
            pooling_layer(qubits[2*i], qubits[(2*i+1)%len_qubits], weights[15*(2*j+1):15*(2*j+2)])

        qub = []
        for i in range(len_qubits):
            if i%2 == 1:
                qub.append(qubits[i])
                
        qubits = qub
    
    convolutional_layer(qubits[0], qubits[1], weights[15*(2*layers-2):15*(2*layers-1)])
    
    return qml.expval(Z(qubits[0])), qml.expval(Z(qubits[1])), qml.expval(Z(qubits[0]) @ Z(qubits[1]))

# dev_draw = qml.device("qiskit.aer", wires=nqubits)
dev = qml.device("default.qubit", wires=nqubits)

# cnn_draw = qml.QNode(cnn, dev_draw)
cnn_circuit = qml.QNode(cnn_circuit, dev, interface="autograd")


def cnn(weights, state_ini):
    z0, z1, zz01 = cnn_circuit(weights, state_ini)

    proj_00 = (1+zz01+z0+z1)/4
    proj_01 = (1-zz01-z0+z1)/4
    proj_10 = (1-zz01+z0-z1)/4
    proj_11 = (1+zz01-z0-z1)/4

    return np.array([proj_00, proj_01, proj_10, proj_11])

In [11]:
def variational_classifier(weights, bias, state_ini):
    return cnn(weights, state_ini) + bias

In [18]:
weights = rand.uniform(0, 2*np.pi, nweights)
drawer = qml.draw(cnn_circuit)
print(drawer(weights, gs_list[0]))

# z0, z1, zz01 = cnn_draw(gs_list[0], weights)
# print(z0, z1, zz01)
# dev_draw._circuit.draw(output="mpl")

0: ─╭|Ψ⟩──U3(4.01,1.23,5.31)──U3(1.15,1.18,4.90)─╭X──RZ(1.18)─╭●───────────╭X──U3(0.45,4.01,5.92)
1: ─├|Ψ⟩─────────────────────────────────────────╰●──RY(0.47)─╰X──RY(1.43)─╰●──U3(4.01,1.23,5.31)
2: ─├|Ψ⟩──U3(4.01,1.23,5.31)──U3(1.15,1.18,4.90)─╭X──RZ(1.18)─╭●───────────╭X──U3(0.45,4.01,5.92)
3: ─╰|Ψ⟩─────────────────────────────────────────╰●──RY(0.47)─╰X──RY(1.43)─╰●──U3(4.01,1.23,5.31)

───U3(3.44,2.88,4.71)──────────────────────────────────────────────────────────────────────╭●
───U3(1.15,1.18,4.90)─╭X──RZ(1.18)─╭●───────────╭X──U3(0.45,4.01,5.92)──U3(3.44,2.88,4.71)─│─
───U3(3.44,2.88,4.71)─╰●──RY(0.47)─╰X──RY(1.43)─╰●─────────────────────────────────────────│─
───U3(1.15,1.18,4.90)──────────────────────────────────────────────────────────────────────╰X

───RY(0.47)─╭X──RY(1.43)─╭●──U3(2.09,5.54,1.56)──U3(4.70,2.52,0.91)─╭X──RZ(3.19)─╭●───────────╭X
────────────│────────────│──────────────────────────────────────────╰●──RY(5.26)─╰X──RY(4.83)─╰●
────────────│────────────│───U3(2.09

# Loss and accuracy

In [14]:
def loss(weights, bias, ground_states, labels):
    cost = 0
    
    for j in range(len(labels)):

        proj_00, proj_01, proj_10, proj_11 = variational_classifier(weights, bias, ground_states[j])
        
        if labels[j] == 0:
            cost += proj_00
        elif labels[j] == 1:
            cost += proj_01
        elif labels[j] == 2:
            cost += proj_10
        else:
            cost += proj_11

        # adding extra terms to equalize incorrect classes might help in the optimization, 
        # although it is not required
        """
        if labels[j] == 0:
            cost += proj_00 + ((proj_01-proj_10)**2 + (proj_01-proj_11)**2 + (proj_10-proj_11)**2)/3
        elif labels[j] == 1:
            cost += proj_01 + ((proj_00-proj_10)**2 + (proj_00-proj_11)**2 + (proj_10-proj_11)**2)/3
        elif labels[j] == 2:
            cost += proj_10 + ((proj_00-proj_01)**2 + (proj_00-proj_11)**2 + (proj_01-proj_11)**2)/3
        else:
            cost += proj_11 + ((proj_00-proj_01)**2 + (proj_00-proj_10)**2 + (proj_01-proj_10)**2)/3
        """

    return cost/len(labels)

In [15]:
def pred_acc(weights, bias, ground_states, labels):
    accuracy_data = 0
    predictions = []

    for j in range(len(labels)):
        projectors = variational_classifier(weights, bias, ground_states[j])
        pred = np.argmin(projectors)

        accuracy_data += 1 if pred == labels[j] else 0
        predictions.append(pred)
            
    return predictions, accuracy_data*100/len(labels)

# Training

In [21]:
if not random:
    rng = rand.RandomState(0)

# Splitting the data into training and validation
num_train = int(perc_train * num_data)
index = rng.permutation(range(num_data))
gs_train = gs_list[index[:num_train]]
labels_train = labels_list[index[:num_train]]
gs_val = gs_list[index[num_train:]]
labels_val = labels_list[index[num_train:]]

# We need these later for plotting
j_train = j_list[index[:num_train]]
j_val = j_list[index[num_train:]]

# weights and bias initialization
weights_init = np.random.uniform(0, max_weight_init, nweights, requires_grad=True)
bias_init = np.array([0.0]*4, requires_grad=True)

# train the variational classifier
opt = NesterovMomentumOptimizer(stepsize)
weights = []
bias = []
w = weights_init
b = bias_init
for it in range(num_iters): #60

    # Update the weights by one optimizer step
    batch_index = np.random.randint(0, num_train, (batch_size,))
    gs_train_batch = gs_train[batch_index]
    labels_train_batch = labels_train[batch_index]
    
    w, b, _, _ = opt.step(loss, w, b, gs_train_batch, labels_train_batch)
    weights.append(w)
    bias.append(b)
    
    # Compute predictions and accuracy on train and validation set
    pred_train, acc_train = pred_acc(w, b, gs_train, labels_train)
    pred_val, acc_val = pred_acc(w, b, gs_val, labels_val)

    print(
        "Iter: {:5d} | Loss: {:0.7f} | Acc train: {:0.7f} | Acc validation: {:0.7f}"
        "".format(it + 1, loss(w, b, gs_list, labels_list), acc_train, acc_val)
    )

Iter:     1 | Loss: 0.3526673 | Acc train: 9.0909091 | Acc validation: 0.0000000
Iter:     2 | Loss: 0.3462436 | Acc train: 9.0909091 | Acc validation: 0.0000000
Iter:     3 | Loss: 0.3355559 | Acc train: 9.0909091 | Acc validation: 0.0000000
Iter:     4 | Loss: 0.3227274 | Acc train: 9.0909091 | Acc validation: 0.0000000
Iter:     5 | Loss: 0.3067356 | Acc train: 9.0909091 | Acc validation: 0.0000000
Iter:     6 | Loss: 0.2878650 | Acc train: 0.0000000 | Acc validation: 0.0000000
Iter:     7 | Loss: 0.2680066 | Acc train: 0.0000000 | Acc validation: 0.0000000
Iter:     8 | Loss: 0.2465625 | Acc train: 0.0000000 | Acc validation: 0.0000000
Iter:     9 | Loss: 0.2235251 | Acc train: 9.0909091 | Acc validation: 0.0000000
Iter:    10 | Loss: 0.1986237 | Acc train: 9.0909091 | Acc validation: 0.0000000
Iter:    11 | Loss: 0.1722219 | Acc train: 9.0909091 | Acc validation: 25.0000000
Iter:    12 | Loss: 0.1425949 | Acc train: 18.1818182 | Acc validation: 25.0000000
Iter:    13 | Loss: 0.109

In [ ]:
plot_iter = 100 #it+1

plt.figure()
cm = plt.cm.RdBu

# make data for decision regions
xx, yy = np.meshgrid(np.linspace(-1.1, 1.1, 20), np.linspace(-1.1, 1.1, 20))
X_grid = [np.array([x, y]) for x, y in zip(xx.flatten(), yy.flatten())]

# preprocess grid points like data inputs above
padding = 0.3 * np.ones((len(X_grid), 1))
X_grid = np.c_[np.c_[X_grid, padding], np.zeros((len(X_grid), 1))]  # pad each input
normalization = np.sqrt(np.sum(X_grid ** 2, -1))
X_grid = (X_grid.T / normalization).T  # normalize each input
features_grid = np.array(
    [get_angles(x) for x in X_grid]
)  # angles for state preparation are new features
predictions_grid = [variational_classifier(weights[plot_iter-1], bias[plot_iter-1], f) for f in features_grid]
Z = np.reshape(predictions_grid, xx.shape)

# plot decision regions
cnt = plt.contourf(
    xx, yy, Z, levels=np.arange(-1, 1.1, 0.1), cmap=cm, alpha=0.8, extend="both"
)
plt.contour(
    xx, yy, Z, levels=[0.0], colors=("black",), linestyles=("--",), linewidths=(0.8,)
)
plt.colorbar(cnt, ticks=[-1, 0, 1])

# plot data
plt.scatter(
    X_train[:, 0][Y_train == 1],
    X_train[:, 1][Y_train == 1],
    c="b",
    marker="o",
    edgecolors="k",
    label="class 1 train",
)
plt.scatter(
    X_val[:, 0][Y_val == 1],
    X_val[:, 1][Y_val == 1],
    c="b",
    marker="^",
    edgecolors="k",
    label="class 1 validation",
)
plt.scatter(
    X_train[:, 0][Y_train == -1],
    X_train[:, 1][Y_train == -1],
    c="r",
    marker="o",
    edgecolors="k",
    label="class -1 train",
)
plt.scatter(
    X_val[:, 0][Y_val == -1],
    X_val[:, 1][Y_val == -1],
    c="r",
    marker="^",
    edgecolors="k",
    label="class -1 validation",
)

plt.legend()
plt.axis('square')
plt.show()
